In [4]:
import os
import re
import pandas as pd

def get_filelist(dir): #读取所有txt
    FileList = []
    for home,dirs,files in os.walk(path):
        for filename in files:
            FileList.append(os.path.join(home,filename))
    return FileList

def filter_str(desstr, restr=' '):
    # 过滤除中英文以外的其他字符
    res = re.compile("[^\\u4e00-\\u9fa5^a-z^A-Z]")
    return res.sub(restr, desstr)

def cutEmail(filename):
    with  open(filename, "r",encoding='GB2312', errors='ignore') as f:
        lines=f.read()
        return filter_str(lines)

In [33]:
path='./ham'
textlist=[]
for hamemail in get_filelist(dir):
    textlist.append(['ham',cutEmail(hamemail),0])

path='./spam'
for hamemail in get_filelist(dir):
    textlist.append(['spam',cutEmail(hamemail),1])    
df=pd.DataFrame(textlist,columns =['HamSpam', 'text', 'Value'])


In [34]:
df

,HamSpam,text,Value
0,ham,Subject re rankings thank you,0
1,ham,Subject re visit to enron vince dec ...,0
2,ham,Subject research group move to the th floo...,0
3,ham,Subject christmas baskets the christmas baske...,0
4,ham,Subject japan candidate vince i spoke with ...,0
...,...,...,...
5852,spam,Subject tetm interest ha ht...,1
5853,spam,Subject localized software all languages av...,1
5854,spam,Subject instant branded software download sof...,1
5855,spam,Subject localized software all languages av...,1


In [35]:
#计算词频
from sklearn.feature_extraction.text import CountVectorizer
vectorizer1 = CountVectorizer(binary=True)

In [36]:
X = vectorizer1.fit_transform(df['text'])
y = df['Value']

In [46]:
#分开训练和湔试样本
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=20)
print ("训练数据中的样本个数: ", X_train.shape[0], "测试数据中的样本个数: ", X_test.shape[0])


训练数据中的样本个数:  4685 测试数据中的样本个数:  1172


In [48]:
#选取多项式分布贝叶斯进行建模
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
clf = MultinomialNB(alpha=1, fit_prior=True)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("accuracy on test data: ", accuracy_score(y_test, y_pred))
print("recall on test data:   ",recall_score(y_test, y_pred))
print("precision on test data:",precision_score(y_test, y_pred))

accuracy on test data:  0.985494880546075
recall on test data:    0.9723926380368099
precision on test data: 0.9753846153846154


In [13]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred, labels=[0, 1])

array([[867,   3],
       [  8, 294]])